In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
prices = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sell_prices.csv")

In [ ]:
prices['id'] = prices['store_id'] + "_" + prices['item_id']

In [ ]:
cal = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")
cal = cal[['wm_yr_wk', 'date', 'd']]
cal['d'] = cal['d'].str.replace("d_", "").astype(int)

In [ ]:
prices = prices.merge(cal[['wm_yr_wk', 'd']].groupby(['wm_yr_wk'])[['d']].min().reset_index(), on = 'wm_yr_wk')

In [ ]:
prices = prices.sort_values(['id', 'd'])

In [ ]:
launch_date = prices.groupby(['id'])[['d']].min().reset_index()

In [ ]:
launch_date.head()

In [ ]:
import gc
gc.collect()

In [ ]:
prices = prices.drop(['d', 'store_id', 'item_id'], axis= 1)

In [ ]:
prices.head()

In [ ]:
cal = cal.drop('date', axis = 1)
cal.head()

In [ ]:
prices = prices.merge(cal, on = 'wm_yr_wk')

In [ ]:
prices = prices.drop(['wm_yr_wk'], axis = 1)

In [ ]:
prices.head()

In [ ]:
prices = prices.pivot(index = 'id', columns = 'd', values = 'sell_price')

In [ ]:
# Remove for eventual test set
prices = prices.iloc[:,:-28]

In [ ]:
prices.reset_index().to_csv("m5_prices_wide.csv", index = False)

In [ ]:
launch_date.to_csv("m5_launch_date.csv", index = False)

In [ ]:
cal = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/calendar.csv")

In [ ]:
print(cal['event_name_1'].unique())
print(cal['event_name_2'].unique())
print(cal['event_type_1'].unique())
print(cal['event_type_2'].unique())

In [ ]:
weekends = cal[['wday']]

In [ ]:
weekends[(weekends['wday'] == 1) | (weekends['wday'] == 2)] = 1
weekends[(weekends['wday'] != 1) & (weekends['wday'] != 2)] = 0

In [ ]:
weekends.columns = ['weekend']

# Remove for eventual set
weekends = weekends.iloc[:-28]

In [ ]:
weekends.to_csv("m5_weekends.csv", index = False)

In [ ]:
sales = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv", usecols = [0,1,2,3,4,5])

In [ ]:
snap_CA = cal['snap_CA'].values
snap_WI = cal['snap_WI'].values
snap_TX = cal['snap_TX'].values

In [ ]:
snap_WI

In [ ]:
np.sum(sales['state_id'] == "CA")

In [ ]:
snap_CA_m = np.repeat(snap_CA.reshape(-1,1), repeats = np.sum(sales['state_id'] == "CA"), axis = 1).transpose()
snap_TX_m = np.repeat(snap_TX.reshape(-1,1), repeats = np.sum(sales['state_id'] == "TX"), axis = 1).transpose()
snap_WI_m = np.repeat(snap_WI.reshape(-1,1), repeats = np.sum(sales['state_id'] == "WI"), axis = 1).transpose()
snap = np.concatenate([snap_CA_m, snap_TX_m, snap_WI_m], axis = 0)

In [ ]:
snap = pd.DataFrame(snap)

In [ ]:
# remove for final set
snap.iloc[:, :-28].to_csv("snap.csv", index = False)

In [ ]:
list_of_hols = list(set(np.append(cal['event_name_1'].unique(), cal['event_name_2'].unique())))[1:]

In [ ]:
holidays = cal.loc[(cal['event_name_1'] == list_of_hols[0]) | (cal['event_name_2'] == list_of_hols[0]), ['date', 'event_name_1', 'date','event_name_2']]

In [ ]:
for i in list_of_hols[1:]:
    holidays = holidays.append(cal.loc[(cal['event_name_1'] == i) | (cal['event_name_2'] == i), ['date', 'event_name_1', 'date','event_name_2']])

In [ ]:
holidays_m = holidays.iloc[:,:2].rename(columns = {'event_name_1':'holiday'}).append(holidays.iloc[:,2:].rename(columns = {'event_name_2':'holiday'}))

In [ ]:
holidays_m = holidays_m.dropna()

In [ ]:
holidays_m = holidays_m[['holiday', 'date']]
holidays_m.columns = ['holiday', 'ds']
holidays_m['lower_window'] = 0
holidays_m['upper_window'] = 0

In [ ]:
holidays_m.loc[holidays_m['holiday'] == "Thanksgiving",'upper_window'] = 1
holidays_m.loc[holidays_m['holiday'] == "Christmas",'upper_window'] = 1
holidays_m.loc[holidays_m['holiday'] == "Easter",'lower_window'] = -2

In [ ]:
holidays_m['ds'] = pd.to_datetime(holidays_m['ds'])

In [ ]:
holidays_m.to_csv("m5_holidays.csv", index = False)

In [ ]:
df = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sample_submission.csv")
df2 = pd.DataFrame(np.zeros([30490, 1913])).astype(int)

In [ ]:
df3 = df.drop(['id'], axis = 1)

In [ ]:
df3.columns = list(np.arange(1914, 1914+28))

In [ ]:
df3

In [ ]:
df2.columns = df2.columns + 1

In [ ]:
df2

In [ ]:
df_sample = pd.concat([df[['id']].head(30490), df2.astype(int), df3.head(30490)], axis = 1)

In [ ]:
df_sample.to_csv("holder.csv", index = False)